In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet101
import os
import cv2
from numpy import asarray

Using TensorFlow backend.


In [3]:
### Organizing classes into training, validation, and test
print("Organizing datasets")
masked_imgs = os.listdir('datasets/compiled/with_mask_copy')
no_mask_imgs = os.listdir('datasets/compiled/without_mask_copy')

i = 0
for img in masked_imgs:
	if i < int(0.80*len(masked_imgs)):
		os.rename('datasets/compiled/with_mask_copy/' + img, 'datasets/compiled/train/mask/' + img)
		i += 1
	else:
		os.rename('datasets/compiled/with_mask_copy/' + img, 'datasets/compiled/validation/mask/' + img)
		i += 1

j = 0
for img in no_mask_imgs:
	if j < int(0.80*len(no_mask_imgs)):
		os.rename('datasets/compiled/without_mask_copy/' + img,'datasets/compiled/train/no-mask/' + img)
		j += 1
	else:
		os.rename('datasets/compiled/without_mask_copy/' + img, 'datasets/compiled/validation/no-mask/' + img)
		j += 1

print("Images reorganized")

Organizing datasets
Images reorganized


In [12]:
epochs = 1
batch_size = 16

In [4]:
train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="nearest",
    horizontal_flip=True,
    rescale=1./255)

validation_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="nearest",
    horizontal_flip=True,
    rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
        batch_size=batch_size,
		directory='datasets/compiled/train/',
        target_size=(224, 224), 
        classes = ['mask','no-mask'],
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        batch_size=batch_size,
        directory='datasets/compiled/validation/',
        target_size=(224, 224), 
        classes = ['mask','no-mask'],
        class_mode='categorical')

Found 3065 images belonging to 2 classes.
Found 768 images belonging to 2 classes.


In [6]:
### Pre-trained Model (ResNet101 trained on Imagenet)
model = tf.keras.applications.ResNet101(include_top=False,weights='imagenet',input_shape=(224,224,3),classes=2)

In [7]:
# Transfer Learning
for i in model.layers:
  i.trainable = False

In [8]:
global_avg = tf.keras.layers.GlobalAveragePooling2D()(model.output)
flatten = tf.keras.layers.Flatten()(global_avg)
# drop_out = tf.keras.layers.Dropout(0.4)(flatten)
out = tf.keras.layers.Dense(2,activation='softmax')(flatten)
resnet = tf.keras.Model(inputs=[model.input],outputs=[out])

In [9]:
resnet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [10]:
resnet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=["accuracy"])

In [13]:
history = resnet.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

192/192 [==============================] - 423s 2s/step - loss: 0.6828 - accuracy: 0.5834 - val_loss: 0.7257 - val_accuracy: 0.5117


In [17]:
test_img = np.asarray('datasets/compiled/train/mask/0_0_006ajtxFly1g5eb2wt4jqj30u00u0myv.jpg')
test_img.resize((224,224))
print(resnet.predict('datasets/compiled/train/mask/0_0_006ajtxFly1g5eb2wt4jqj30u00u0myv.jpg'))

IndexError: list index out of range

In [28]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [1]:
labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
cv2.namedWindow("COVID Mask Detection Video Feed")
webcam = cv2.VideoCapture(0) 

classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    rval, im = webcam.read()
    im=cv2.flip(im,1,1)
    
    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(224,224))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,224,224,3))
        reshaped = np.vstack([reshaped])
        result=resnet.predict(reshaped)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    if im is not None:   
        cv2.imshow('COVID Mask Detection Video Feed', im)
    key = cv2.waitKey(10)
    
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
        
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined